# Supplementary Analysis: Interaction Effects (3-Way ANOVA)

This notebook executes a 3-way ANOVA to analyze the interaction effects of **Sentiment** (Positive/Neutral/Negative), **Agent** (e.g., Copilot, Devin), and **User Type** (Bot/User) on the efficiency of pull requests (measured by iterations per hour).

**Hypothesis**: There are significant interaction effects between the coding agent used, the sentiment of the review feedback, and the type of reviewer (human vs. bot).


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from huggingface_hub import hf_hub_download
import traceback

# Set plot style
sns.set_theme(style="whitegrid")

# Ensure VADER lexicon is downloaded
try:
    nltk.data.find('sentiment/vader_lexicon.zip')
except LookupError:
    print("Downloading VADER lexicon...")
    nltk.download('vader_lexicon')


## 1. Load Data
We load the necessary datasets from the Hugging Face repository.


In [ ]:
print("Loading datasets...")

try:
    # Download files from Hugging Face to local cache
    dataset_name = "hao-li/AIDev"
    
    # Download all required files
    files_to_download = [
        "pull_request.parquet",
        "pr_reviews.parquet",
        "pr_timeline.parquet",
    ]
    
    file_paths = {}
    for filename in files_to_download:
        # print(f"  Downloading {filename}...")
        file_paths[filename] = hf_hub_download(repo_id=dataset_name, filename=filename, repo_type="dataset")
    
    print("Loading parquet files into dataframes...")
    
    # Basic PR data
    pr_df = pd.read_parquet(file_paths["pull_request.parquet"])
    
    # Reviews
    pr_reviews_df = pd.read_parquet(file_paths["pr_reviews.parquet"])
    
    # Events/Timeline
    pr_timeline_df = pd.read_parquet(file_paths["pr_timeline.parquet"])
    
    print("✓ All datasets loaded successfully!")

except Exception as e:
    print(f"Error loading datasets: {e}")
    traceback.print_exc()
    pr_df, pr_timeline_df, pr_reviews_df = None, None, None


## 2. Preprocess Data
We calculate the lifecycle of each PR in hours and the number of iteration cycles.


In [ ]:
if pr_df is not None:
    print("Preprocessing PR data...")
    # Filter for closed PRs
    closed_prs = pr_df[pr_df["state"] == "closed"].copy()
    
    # Convert timestamps
    closed_prs["created_at"] = pd.to_datetime(closed_prs["created_at"])
    closed_prs["closed_at"] = pd.to_datetime(closed_prs["closed_at"])
    
    # Calculate lifecycle in hours
    closed_prs["lifecycle_hours"] = (closed_prs["closed_at"] - closed_prs["created_at"]).dt.total_seconds() / 3600
    
    pr_lifecycle = closed_prs[["id", "state", "lifecycle_hours"]]
    print(f"Processed {len(pr_lifecycle)} closed PRs.")


In [ ]:
if pr_timeline_df is not None:
    print("Calculating iteration cycles...")
    pr_iterations = (
        pr_timeline_df
        .groupby("pr_id")
        .size()
        .reset_index(name="iteration_cycles")
    )
    print(f"Calculated iterations for {len(pr_iterations)} PRs.")


## 3. Sentiment Analysis
We use VADER to analyze the sentiment of PR reviews, classifying them into Positive, Negative, or Neutral.


In [ ]:
if pr_reviews_df is not None:
    print("Analyzing sentiment...")
    # Extract relevant columns and drop missing bodies
    reviews = pr_reviews_df[["pr_id", "body"]].dropna().copy()
    
    sia = SentimentIntensityAnalyzer()
    
    def get_compound_score(text):
        return sia.polarity_scores(text)["compound"]
    
    # Calculate sentiment for each review
    print("Calculating VADER scores...")
    reviews["compound_score"] = reviews["body"].apply(get_compound_score)
    
    # Aggregate by PR (Mean score)
    print("Aggregating sentiment per PR...")
    pr_sentiment = reviews.groupby("pr_id")["compound_score"].mean().reset_index()
    
    # Classify sentiment
    def classify_sentiment(score):
        if score > 0.05:
            return "Positive"
        elif score < -0.05:
            return "Negative"
        else:
            return "Neutral"
            
    pr_sentiment["sentiment_category"] = pr_sentiment["compound_score"].apply(classify_sentiment)
    pr_sentiment.columns = ["id", "mean_compound_score", "sentiment_category"]
    
    print(pr_sentiment.head())


## 4. Construct Final Dataset
We merge the lifecycle, iterations, sentiment, and agent/user information into a single DataFrame for analysis.


In [ ]:
if pr_df is not None and pr_timeline_df is not None and pr_reviews_df is not None:
    print("Merging data...")
    
    # 1. Merge Lifecycle and Iterations
    merged_df = pd.merge(
        pr_lifecycle,
        pr_iterations,
        left_on="id",
        right_on="pr_id",
        how="left"
    )
    
    # 2. Merge Sentiment
    # We use inner join to keep only PRs that have reviews/sentiment data
    final_df = pd.merge(
        merged_df,
        pr_sentiment,
        on="id",
        how="inner" 
    )
    
    # 3. Add Agent info (from closed_prs)
    # Note: 'agent' column is assumed to exist in the loaded pull_request.parquet
    merged_final_df = final_df.merge(
        closed_prs[["id", "agent"]], 
        left_on="id", 
        right_on="id", 
        how="left"
    )

    # 4. Add User Type info (from pr_reviews_df)
    # Note: merging on pr_id and user_type, dropping duplicates to avoid explosion if multiple types exist
    formatted_reviews = pr_reviews_df[["pr_id", "user_type"]].drop_duplicates()
    
    merged_final_df = merged_final_df.merge(
        formatted_reviews, 
        left_on="id", 
        right_on="pr_id", 
        how="left"
    )

    # Calculate Efficiency Metric
    merged_final_df["iteration_per_hour"] = merged_final_df["iteration_cycles"] / merged_final_df["lifecycle_hours"]

    print("Final merged dataset ready.")
    print(merged_final_df.head())


## 5. Statistical Analysis (3-Way ANOVA)
We perform a 3-way ANOVA to test the effects of Agent, User Type, and Sentiment Category on `iteration_per_hour`.


In [ ]:
# Fit the model
model = ols('iteration_per_hour ~ C(agent) * C(user_type) * C(sentiment_category)', data=merged_final_df).fit()

# Perform ANOVA
anova_table = sm.stats.anova_lm(model, typ=2)

# Display results
anova_table


## 6. Post-hoc Analysis (Tukey HSD)
We perform Tukey HSD tests to determine which specific groups differ significantly.


In [ ]:
# Compare agents
print("--- Tukey HSD: Agent ---")
tukey_agent = pairwise_tukeyhsd(endog=merged_final_df['iteration_per_hour'], groups=merged_final_df['agent'], alpha=0.05)
print(tukey_agent)

# Compare user types
print("\n--- Tukey HSD: User Type ---")
tukey_user = pairwise_tukeyhsd(endog=merged_final_df['iteration_per_hour'], groups=merged_final_df['user_type'], alpha=0.05)
print(tukey_user)

# Compare sentiment categories
print("\n--- Tukey HSD: Sentiment Category ---")
tukey_sentiment = pairwise_tukeyhsd(endog=merged_final_df['iteration_per_hour'], groups=merged_final_df['sentiment_category'], alpha=0.05)
print(tukey_sentiment)
